This example notebook uses the averaging functions found ins the diff_classifier msd module to find average msd profiles over input msd datasets using precision-weighted averaging. Precision is the inverse of the standard squared error. This increases the contribution of videos that have many particles and more homogeneous datasets to the final calculated MSD.

In [ ]:
import numpy as np
import diff_classifier.aws as aws
import diff_classifier.msd as msd

In [ ]:
folder = 'Tissue_Studies/09_11_18_Regional'
bucket = 'ccurtis.data'
#experiment = 'test' #Used for naming purposes. Should exclude XY and well information

#vids = 2
to_track = []
frames = 651
fps = 100.02
umppx = 0.07

vids = 15
types = ['PEG']
pups = [2, 3]
slices = [1, 2, 3]
for typ in types:
    for pup in pups:
        for slic in slices:
            for num in range(1, vids+1):
                #to_track.append('100x_0_4_1_2_gel_{}_bulk_vid_{}'.format(vis, num))
                to_track.append('{}_P{}_S{}_XY{}'.format(typ, pup, slic, '%02d' % num))

In [ ]:
to_track

In [ ]:
experiment = 'PEG'
slices, bins, well_names = binning(to_track, wells = wells, prefix=experiment)

geomean = {}
gSEM = {}
for sample_name in to_track[28:]:
    # Users can toggle between using pre-calculated geomean files and calculating new values by commenting out the relevant
    # lines of code within the for loop.
    #aws.download_s3('{}/geomean_{}.csv'.format(folder, sample_name), 'geomean_{}.csv'.format(sample_name), bucket_name=bucket)
    #aws.download_s3('{}/geoSEM_{}.csv'.format(folder, sample_name), 'geoSEM_{}.csv'.format(sample_name), bucket_name=bucket)
    #geomean[sample_name] = np.genfromtxt('geomean_{}.csv'.format(sample_name))
    #gSEM[sample_name] = np.genfromtxt('geoSEM_{}.csv'.format(sample_name))
    
    aws.download_s3('{}/msd_{}.csv'.format(folder, sample_name), 'msd_{}.csv'.format(sample_name), bucket_name=bucket)
    geomean[sample_name], gSEM[sample_name] = msd.geomean_msdisp(sample_name, umppx=umppx, fps=fps,
                                                                 remote_folder=folder, bucket=bucket)
    print('Done with {}'.format(sample_name))

In [ ]:
experiment = 'PEG'
slices, bins, well_names = binning(to_track, wells = wells, prefix=experiment)

In [ ]:
geo_slices = {}
gSEM_slices = {}

w_slices = {}
wo_slices = {}
w_holder = {}
wh = {}
gh = {}

for key in bins:
    print(key)
    wo_slices[key], wh[key] = precision_weight(bins[key], gSEM)
    geodata = precision_averaging(bins[key], geoM2xy, gSEM, wo_slices[key],
                                  bucket=bucket, folder=folder, experiment=key)
    geo_slices[key] = geodata.geomean
    gSEM_slices[key] = geodata.gSEM
    
weights, wh1 = precision_weight(well_names, gSEM_slices)
geo, gS, w_holder1, gh1 = precision_averaging(well_names, geo_slices, gSEM_slices, weights, bucket=bucket, folder=folder, experiment=experiment)

In [ ]:
weights, wh1 = msd.precision_weight(to_track, gSEM)
geodata = msd.precision_averaging(to_track, geomean, gSEM, weights,
                                  bucket=bucket, folder=folder, experiment='gel_12_percent_bulk')

Note that in cases where two or more averaging steps are needed (for instance, if the user takes 5 videos per well with a total of four wells), averaging steps can be performed consecutively. the msd.binning function is a helpful tool by defining bins over which to average for multi-step averaging.

In [ ]:
msd.plot_all_experiments(to_track, yrange=(10**-3, 10**-1), bucket=bucket, folder=folder)

In [ ]:
msd.plot_all_experiments(['gel_04_percent_bulk', 'gel_12_percent_bulk'], yrange=(10**-3, 10**-1), bucket=bucket, folder=folder)